Script to alter the til tracking numpy arrays if an error has occurred

In [1]:
# Standard packages
import tempfile
import warnings
import urllib
import shutil
import os
# Less standard, but still pip- or conda-installable
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
#import rasterio
import re
import rtree
import shapely
import pickle
import data_eng.az_proc as ap
#from cartopy import crs
import collections

In [2]:
import PIL
print('PIL',PIL.__version__)
from PIL import Image
Image.MAX_IMAGE_PIXELS = 999999999999

PIL 8.3.1


Specify the parameters to download images

In [13]:
parent_directory = "C:\chips_allocation"
annotation_directory = "Check"
number_of_tiles = 100
tiles_remaining = "tile_name_tile_url_remaining_expanded.npy"
tiles_labeled = "tile_name_tile_url_labeled.npy"

Load the annotator class

In [18]:
dist = ap.annotator(annotation_directory) #create the processing class
dist.state_dcc_directory(parent_directory)
dist.number_of_tiles(number_of_tiles)
dist.get_tile_urls(tiles_remaining)
dist.make_subdirectories()
dist.download_images()
#dist.tile_rename()

Unlabeled Tiles (205, 2)
0
Bypassing download of already-downloaded file m_3108961_se_16_060_20181103_20190207.tif
1
Bypassing download of already-downloaded file m_3907527_ne_18_060_20180810.tif
2
Bypassing download of already-downloaded file m_3509055_se_15_060_20180803.tif
3
Bypassing download of already-downloaded file m_3807730_se_18_060_20181025.tif
4
Bypassing download of already-downloaded file m_4308663_nw_16_060_20180707_20181211.tif
5
Bypassing download of already-downloaded file m_3311718_sw_11_060_20180822_20190209.tif
6
Bypassing download of already-downloaded file m_4108721_se_16_060_20180707_20190221.tif
7
Bypassing download of already-downloaded file m_3510132_se_14_060_20180917.tif
8
Bypassing download of already-downloaded file m_3809038_ne_15_060_20181121_20190206.tif
9
Bypassing download of already-downloaded file m_4406826_ne_19_060_20180720.tif
10
Bypassing download of already-downloaded file m_3310216_ne_13_060_20180923.tif
11
Bypassing download of already-downl

100% (580540980 of 580540980) |##########| Elapsed Time: 0:00:08 Time:  0:00:08


...done, 580540980 bytes.
96


100% (598169372 of 598169372) |##########| Elapsed Time: 0:00:08 Time:  0:00:08


...done, 598169372 bytes.
97


...

100% (579522759 of 579522759) |##########| Elapsed Time: 0:00:07 Time:  0:00:07


...done, 579522759 bytes.
99


100% (587501609 of 587501609) |##########| Elapsed Time: 0:00:11 Time:  0:00:11


...done, 587501609 bytes.


['C:\\chips_allocation/Check\\tiles\\https_naipblobs.blob.core.windows.net_naip_v002_ms_2018_ms_060cm_2018_31089_m_3108961_se_16_060_20181103_20190207.tif',
 'C:\\chips_allocation/Check\\tiles\\https_naipblobs.blob.core.windows.net_naip_v002_de_2018_de_060cm_2018_39075_m_3907527_ne_18_060_20180810.tif',
 'C:\\chips_allocation/Check\\tiles\\https_naipblobs.blob.core.windows.net_naip_v002_tn_2018_tn_060cm_2018_35090_m_3509055_se_15_060_20180803.tif',
 'C:\\chips_allocation/Check\\tiles\\https_naipblobs.blob.core.windows.net_naip_v002_va_2018_va_060cm_2018_38077_m_3807730_se_18_060_20181025.tif',
 'C:\\chips_allocation/Check\\tiles\\https_naipblobs.blob.core.windows.net_naip_v002_mi_2018_mi_060cm_2018_43086_m_4308663_nw_16_060_20180707_20181211.tif',
 'C:\\chips_allocation/Check\\tiles\\https_naipblobs.blob.core.windows.net_naip_v002_ca_2018_ca_060cm_2018_33117_m_3311718_sw_11_060_20180822_20190209.tif',
 'C:\\chips_allocation/Check\\tiles\\https_naipblobs.blob.core.windows.net_naip_v002_

## Code to Fix arrays 

Fix arrays when tiles are not properly allocated

In [ ]:
N =   #specify the number of tiles that were not properly allocated
#load the array
remaining = np.load('tile_name_tile_url_remaining_expanded.npy')
labeled = np.load("tile_name_tile_url_labeled.npy")
#Save the ammended arrays
np.save('tile_name_tile_url_labeled',labeled[:-N])
np.save('tile_name_tile_url_remaining_expanded',np.concatenate((labeled[-N:], remaining)))

Fix arrays when a tile cannot be found (HTTP Error 404: The specified blob does not exist.)

In [17]:
row_to_remove = 95

#dist.tile_name_tile_url_tiles_for_annotators[row_to_remove]

error_downloading = np.concatenate((np.load('tile_name_tile_url_error_downloading.npy'),
                                    dist.tile_name_tile_url_tiles_for_annotators[row_to_remove]))
np.save('tile_name_tile_url_error_downloading.npy', error_downloading)

#Remove the rows with errors
print(np.load('tile_name_tile_url_remaining_expanded.npy').shape)
np.save('tile_name_tile_url_remaining_expanded',np.delete(np.load(tiles_remaining), row_to_remove, axis = 0))
print(np.load('tile_name_tile_url_remaining_expanded.npy').shape)
row_to_remove = np.nan

(206, 2)
(205, 2)


In [ ]:
#recover the unlabeled images from the dist
## np.save('tile_name_tile_url_remaining_expanded',dist.tile_name_tile_url_unlabeled)
#dist.tile_name_tile_url_tiles_for_annotators

In [ ]:
#dist.track_tile_annotations(tiles_labeled)
#np.save('tile_name_tile_url_remaining_expanded', dist.tile_name_tile_url_remaining)
#np.save('tile_name_tile_url_labeled', dist.tile_name_tile_url_labeled)

## FIX Crop Script

In [ ]:
from PIL import Image

def crop(path, input, height, width, k, page, area):
    im = Image.open(input)
    imgwidth, imgheight = im.size
    for i in range(0,imgheight,height):
        for j in range(0,imgwidth,width):
            box = (j, i, j+width, i+height)
            a = im.crop(box)
            try:
                o = a.crop(area)
                o.save(os.path.join(path,"PNG","%s" % page,"IMG-%s.png" % k))
            except:
                pass
            k +=1

In [ ]:
dist.tile_names = os.listdir(dist.tiles_dir) #get a list of all of the tiles in tiles directory
for tile_name in dist.tile_names: #index over the tiles in the tiles_dir 
    file_name, ext = os.path.splitext(tile_name) # File name
    print(tile_name)

    print(count)

### Pil

In [ ]:
from PIL import Image 
 
#img = Image.open(r"C:\Users\rapiduser\Downloads\shutterstock_95235265.jpg") 
img = Image.open(r"C:\chips_allocation\Check\tiles\ca_060cm_2018_38121_m_3812128_ne_10_060_20180714.tif")
img.save('test.jpeg')
img = Image.open(r"C:\chips_allocation\Check\tiles\ca_060cm_2018_38121_m_3812128_ne_10_060_20180714.")

left = 0
top = 50
right = 510
bottom = 292
 
img_res = img.crop((left, top, right, bottom)) 
 
img_res.show() 

### Open CV

In [14]:
import cv2
import os
image = cv2.imread(r"C:/chips_allocation/Check/tiles/ca_060cm_2018_38121_m_3812128_ne_10_060_20180714.tif") 
y=0
x=512*19
h=512
w=512
#cropped = image[x:w, y:h]
cropped = image[y:y+h, x:x+w]
print(image.shape)

# Filename
filename = 'test2.jpg'
cv2.imwrite(os.path.join('C:/chips_allocation/Check/tiles',filename),cropped)
cv2.imshow("Cropped", cropped)
cv2.waitKey(0)
# Using cv2.imwrite() method
# Saving the image
#cv2.imwrite(filename, img)


(12220, 9770, 3)


-1

In [ ]:
from PIL import Image
Image.MAX_IMAGE_PIXELS = 999999999999
import os
file_name = "ca_060cm_2018_38121_m_3812128_ne_10_060_20180714"
#tile = Image.open(os.path.join(dist.tiles_dir, tile_name)) #open image
with Image.open(r"C:\chips_allocation\Check\tiles\ca_060cm_2018_38121_m_3812128_ne_10_060_20180714") as tile:
    width, height = tile.size
    item_width = int(512)
    col = int(width/512)+1
    row = int(height/512)+1
    count = 1
    for j in range(0,col):
        for i in range(0,row):
            box = (i*item_width, j*item_width, (i+1)*item_width, (j+1)*item_width)
            chips_save_path = os.path.join(self.chips_dir, file_name+ '_'+ \
                                           str(count).zfill(6) + '.jpg') 
            # The index is a six-digit number like '000023'. 
            #chip = tile.crop(box)
            #chip = chip.save(chips_save_path)
            chip = tile.crop(box).save(chips_save_path)
            count += 1
    print(count)

In [4]:
import numpy as np
import cv2
import os

image = cv2.imread(r"C:/chips_allocation/Check/tiles/ca_060cm_2018_38121_m_3812128_ne_10_060_20180714.tif") 
y=0
x=512*19
h=512
w=512
#cropped = image[x:w, y:h]
cropped = image[y:y+h, x:x+w]

print(image.shape)

# Filename
filename = 'test2.jpg'
cv2.imwrite(os.path.join('C:/chips_allocation/Check/tiles',filename),cropped)
cv2.imshow("Cropped", cropped)
cv2.waitKey(0)
# Using cv2.imwrite() method
# Saving the image
#cv2.imwrite(filename, img)

#image = cv2.imread('C:/chips_allocation/Check/tiles/test2.jpg') 
print(chip_img.width)
print(chip_img.height)

#At the end of the image
if chip_img.width != 512:
    black_width = 512 - chip_img.width
    black_height = 512
    black_image = np.zeros((black_height,black_width,3), np.uint8)
    print("Incorrect Width")
    print(black_img.width)
    print(black_img.height)
    chip_img = np.concatenate([chip_img, blank_image])
    
if chip_img.height != 512:
    black_height = 512 - chip_img.height
    black_width = 512
    black_image = np.zeros((black_height,black_width,3), np.uint8)
    print("Incorrect Height")
    print(black_img.width)
    print(black_img.height)
    chip_img = np.concatenate([chip_img, blank_image])

In [8]:
#blank_image = np.zeros((512,512-42,3), np.uint8)
cv2.imshow('3 Channel Window', 1)
print("image shape: ", blank_image.shape)
cv2.waitKey(0)
cv2.destroyAllWindows()

image shape:  (512, 470, 3)


In [35]:
from PIL import Image


#blackBox = Image.new(im.mode, (im.width, 50), '#000')
#im.paste(blackBox, (0, im.height - blackBox.height))
#for x in range(img.width):
#    for y in range(img.height - 40, img.height):
#        img.putpixel((x, y), (0, 0, 0))

#img.save('test2_bb.png')

42
512


In [40]:
from PIL import Image  
chip_img = Image.open('C:/chips_allocation/Check/tiles/test2.jpg')
print(chip_img.width)
print(chip_img.height)

if chip_img.width != 512:
    black_width = 512 - chip_img.width
    black_height = 512
    black_img = Image.new(mode = "RGB", size = (black_width, black_height))
    print("Incorrect Width")
    print(black_img.width)
    print(black_img.height)
    chip_img+black_img
    
if chip_img.height != 512:
    black_height = 512 - chip_img.height
    black_width = 512
    blackimg = Image.new(mode = "RGB", size = (black_width, black_height))
    print("Incorrect Height")
    print(black_img.width)
    print(black_img.height)

42
512
Incorrect Width
470
512


TypeError: unsupported operand type(s) for +: 'JpegImageFile' and 'Image'

In [ ]:
import numpy as np
import cv2

image = cv2.imread('download.jpg')
y=0
x=0
h=100
w=200
crop = image[y:y+h, x:x+w]
cv2.imshow('Image', crop)
cv2.waitKey(0) 

### Wand

In [7]:
from __future__ import print_function
from wand.image import Image

with Image(filename=r"C:/chips_allocation/Check/tiles/ca_060cm_2018_38121_m_3812128_ne_10_060_20180714.tif") as img:
    print('format =', img.format)
    print('width =', img.width)
    print('height =', img.height)

format = TIFF
width = 306
height = 382
